# Codon optimization for *Phaeodactylum tricornutum*

### Latest update: 17 April 2020

### Raffaela Abbriano (raffaela.abbriano@uts.edu.au)

This code is designed to codon optomize sequences for expression in *Phaeodactylum tricornutum*. Multiple nucleotide sequences can be analyzed if included in one fasta file. Rare codons were determined if %frequency was less than 10% in at least one of the following: Plant codon usage database for *P. tricornutum* (http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=2850); Scala et al. *Plant Physiology* (2002) Supplemental Table IV; Montsant et al. *Plant Physiology* (2005) Supplemental Table IV. Rare codons are replaced with the most adundant codon for that amino acid in *P. tricornutum*. The number of substitutions made for each sequence is reported, and both original and modified sequences are translated and aligned to confirm that no change has been made to the amino acid sequence. New codon optimized nucleotide sequences appear in a new file 'results.fasta' in the same directory.

To run, replace the name of 'mysequences.fasta' with the name of your fasta file. Codons to be substituted can be changed by adding or removing them from the Ptcodons dictionary.

In [5]:
#Import necessary libraries, requires Biopython package
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [6]:
#Rare codon table
Ptcodons = {
    "ATA" : "ATT", "CTA" : "CTC", "TTA" : "TTG", "AGA" : "CGT", 
    "AGG" : "CGT", "CGG" : "CGT", "TCA" : "TCC", "GGG" : "GGA", 
    "GTA" : "GTC"
}

#DNA codon table
protein = {"TTT" : "F", "CTT" : "L", "ATT" : "I", "GTT" : "V", "TTC" : "F", "CTC" : "L", 
           "ATC" : "I", "GTC" : "V", "TTA" : "L", "CTA" : "L", "ATA" : "I", "GTA" : "V", 
           "TTG" : "L", "CTG" : "L", "ATG" : "M", "GTG" : "V", "TCT" : "S", "CCT" : "P", 
           "ACT" : "T", "GCT" : "A", "TCC" : "S", "CCC" : "P", "ACC" : "T", "GCC" : "A",
           "TCA" : "S", "CCA" : "P", "ACA" : "T", "GCA" : "A", "TCG" : "S", "CCG" : "P", 
           "ACG" : "T", "GCG" : "A", "TAT" : "Y", "CAT" : "H", "AAT" : "N", "GAT" : "D", 
           "TAC" : "Y", "CAC" : "H", "AAC" : "N", "GAC" : "D", "TAA" : "*", "CAA" : "Q", 
           "AAA" : "K", "GAA" : "E", "TAG" : "*", "CAG" : "Q", "AAG" : "K", "GAG" : "E",
           "TGT" : "C", "CGT" : "R", "AGT" : "S", "GGT" : "G", "TGC" : "C", "CGC" : "R", 
           "AGC" : "S", "GGC" : "G", "TGA" : "*", "CGA" : "R", "AGA" : "R", "GGA" : "G", 
           "TGG" : "W", "CGG" : "R", "AGG" : "R", "GGG" : "G" 
}

In [8]:
#Make dictionary of sequences with Biopython, change input filename as needed
record_dict = SeqIO.index('mysequences.fasta', 'fasta')

with open('results.fasta', 'w') as f:
    for record in record_dict:
    
        #Convert from byte (biopython format) to string type
        dna_record = record_dict.get_raw(record)
        decoded = dna_record.decode('ASCII')
        decoded_split = decoded.split('\n')
        #First part of decoded sequence is the header, second part is the nucelotide sequence
        dna_seq = decoded_split[1]
        
        #Set blank lists and counter
        optimised = ''
        translated = ''
        optandtrans = ''
        count = 0
        
        #For each codon, test if in Ptcodons; if yes replace w/ value, if no replace w/ self
        for i in range(0, len(dna_seq)-(len(dna_seq)%3), 3):
            sub_dna_seq = dna_seq[i:i+3]
            if sub_dna_seq in Ptcodons:
                #print(sub_dna_seq)
                #print(Ptcodons[sub_dna_seq]) 
                sub_dna_seq = Ptcodons[sub_dna_seq]
                optimised += sub_dna_seq
                count = count + 1
            else:
                optimised += sub_dna_seq
               
            #Use DNA codon table to translate original and optimised sequences
            translated += protein[dna_seq[i:i+3]]
            optandtrans += protein[sub_dna_seq]
            
        #Quality control - check nucleotide and protein alignments
        if dna_seq == optimised:
            print("DNA sequences are identical, no substitutions were made.")
        else:
            print(count,'codon subsitution(s) were made for sequence',decoded_split[0])
            
        if translated == optandtrans:
            print('Translation of sequences are identical.')
        else:
            print('Check sequence translations, sequences do not match.')
            
        aa_alignments = pairwise2.align.globalxx(translated, optandtrans)
        print(format_alignment(*aa_alignments[0]))

        #Print new optimised sequences to output fasta file
        print(decoded_split[0]+'_opt\n'+optimised, file=f)
        